## Day 9:

In [ ]:
import pytest
from icecream import ic

In [ ]:
def load_input(filename):
    with open(filename, 'r') as f_input:
        for line in f_input:
            line = line.strip()
            yield line

In [ ]:
...

In [ ]:
sol = solution_one('input.txt')
print(f"Solution part one: {sol}")